<a href="https://colab.research.google.com/github/BrunoBmassa/Python-Actitivies/blob/main/leitura_escrita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [2]:
import pyspark
import gdown
from pyspark.sql import SparkSession
from pyspark.sql.types import*
from datetime import date

In [3]:
spark= SparkSession.builder.getOrCreate()


In [4]:
#videos-stats
url='https://drive.google.com/file/d/1XAefeuunm6q7o0OEdF--NzEPXTmIG9i5/view?usp=drive_link'


In [5]:
# Extração do arquivo pela URL
file_id = url.split('/')[-2]

In [6]:
# Download do arquivo
output_file = 'videos-stats.csv'
gdown.download(id=file_id, output=output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1XAefeuunm6q7o0OEdF--NzEPXTmIG9i5
To: /content/videos-stats.csv
100%|██████████| 227k/227k [00:00<00:00, 94.0MB/s]


'videos-stats.csv'

In [7]:

# Leitura do arquivo CSV usando spark
df = spark.read.csv(output_file)

In [8]:
#Visualize os primeiros 8 registro do arquivo
df.show(8)


+----+--------------------+-----------+------------+-------+--------+--------+---------+
| _c0|                 _c1|        _c2|         _c3|    _c4|     _c5|     _c6|      _c7|
+----+--------------------+-----------+------------+-------+--------+--------+---------+
|NULL|               Title|   Video ID|Published At|Keyword|   Likes|Comments|    Views|
|   0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0| 135612.0|
|   1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|1758063.0|
|   2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|1564007.0|
|   3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0| 922918.0|
|   4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|1855644.0|
|   5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0| 943119.0|
|   6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047.0|  9367.0|5937790.0|
+----+---------------

In [9]:
#Visualize o esquema do arquivo
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [10]:
#Leia novamente o arquivo inferindo o esquema e visualize o esquema novamente

df = spark.read.csv(output_file, header=True, inferSchema=True)
df.printSchema()


root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: double (nullable = true)
 |-- Comments: double (nullable = true)
 |-- Views: double (nullable = true)



In [11]:
#Salve o arquivo como 'videos-parquet' no formato parquet e adicione o cabeçalho nos dados

df.write.option('header','true').mode("overwrite").parquet('videos-parquet')


In [12]:
#Leia e visualize o arquivo 'videos-parquet' com cabeçalho nos dados;

df = spark.read.parquet('videos-parquet')
df.show()

+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|      Views|
+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|  1855644.0|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0|   943119.0|
|  6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047.0|  9367.0|  5937790.0|
|  7|I bought the STRA...|2TqOmtTAMRY|  2022-08-07|   tech|216935.0| 12605.0|  4782514.0|
|  8|15 Em

In [13]:
#Salve o arquivo do exec. anterior como tabela chamada ‘tb_videos’ no banco de dados default do spark catalog

df.write.option('header','true').mode("overwrite").saveAsTable('tb_videos')

In [14]:
#Liste as tabelas do spark catalog para verificar a tabela

spark.catalog.listTables()

[Table(name='tb_videos', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]

In [15]:
#Utilize o spark SQL para ler a tabela ‘tb_videos’

tab_df = spark.sql('select * from tb_videos')
tab_df.show()

+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|      Views|
+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|  1855644.0|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0|   943119.0|
|  6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047.0|  9367.0|  5937790.0|
|  7|I bought the STRA...|2TqOmtTAMRY|  2022-08-07|   tech|216935.0| 12605.0|  4782514.0|
|  8|15 Em

In [18]:
#Leia o arquivo ‘comments.csv' inferindo o esquema e visualize

url2='https://drive.google.com/file/d/1rW-4ufh6UqafMAeRrH1MVypHkS75lfFM/view?usp=drive_link'

# Extração do arquivo pela URL
file_id2 = url2.split('/')[-2]

# Download do arquivo
output_file2 = 'comments.csv'
gdown.download(id=file_id2, output=output_file2, quiet=False)


#Leitura do arquivo
df = spark.read.csv(output_file2, header=True, inferSchema=True)
df.printSchema()


Downloading...
From: https://drive.google.com/uc?id=1rW-4ufh6UqafMAeRrH1MVypHkS75lfFM
To: /content/comments.csv
100%|██████████| 4.05M/4.05M [00:00<00:00, 174MB/s]


root
 |-- _c0: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Likes: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [19]:
#Salve o arquivo como ‘comments-parquet' no formato parquet e adicione o cabeçalho nos dados

df.write.option('header','true').mode("overwrite").parquet('comments-parquet')
